In [1]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup

# 네이버 영화 댓글
```
1) 페이지 요청하기 (requsets 모듈을 이용해서) => html 페이지를 텍스트로 저장 

2) html 페이지 => soup 객체로 

3) 데이타 => 데이타프레임 

4) 데이타프레임 => csv파일로 저장
```

### soup 필터링 메서드

- find(태그/클래스/아이디) => 첫번쨰 요소
- find_all(class_='클래스명') => 전체 요소 클래스
- find_all('태그명', {'class':'클래스명'}) => 전체 요소

- select(css선택자) => 리스트
- select_one(css선택자) => 첫번쨰 요소 

In [2]:
# 1) requests 모듈 페이지 요청 => html 페이지를 텍스트로 저장 
url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1'
res = requests.get(url)
# 200 이면 성공 
print(res) 
# html 페이지를 텍스트로 저장 
html_str = res.text
print(type(html_str))

<Response [200]>
<class 'str'>


In [3]:
# 2) BeautifulSoup() 이용... html 페이지(str) => soup 객체
#  BeautifulSoup(html페이지소스, 파서(html.parser/lxml/xml))
soup = BeautifulSoup(html_str, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [4]:
# 3) 필요한 요소만 리스트로 저장 
# 필터링 요소 찾기 - 태그 위주 

# td_list = soup.find_all(class_='title')
td_list = soup.select('.title')

In [5]:
print('페이지당 댓글 갯수는?', len(td_list))
print('상위 첫번째 댓글 소스 확인')
td_list[0]

페이지당 댓글 갯수는? 10
상위 첫번째 댓글 소스 확인


<td class="title">
<a class="movie color_b" href="/movie/bi/mi/basic.naver?code=13558">키드</a>
<div class="list_netizen_score">
<span class="st_off"><span class="st_on" style="width:100%">별점 - 총 10점 중</span></span><em>10</em>
</div>
<br/>참으로 아름다운 영화다 
			
			
			
				
				
				
				<a class="report" href="#" onclick="report('tlat****', 'ejLbKJPQwH/dirGHccqIordozOZgwHIkACvwjOvYnVk=', '참으로 아름다운 영화다', '18048385', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
</td>

In [6]:
# 첫번째 요소에 필요한 텍스트만 추출 
# title = td_list[0].find('a').text
title = td_list[0].select_one('a').text # 영화 제목
score = td_list[0].select_one('em').text  # 평점 
# .next_sibling 요소의 다음 형제 요소 찾기 
# br 태그 요소의 다음 요소를 찾아서 공백등 제거 후 저장해줘 
review = td_list[0].select_one('br').next_sibling.strip()

base_url = 'https://movie.naver.com'
link = base_url + td_list[0].select_one('a')['href']
print(title, score, review, link)

키드 10 참으로 아름다운 영화다 https://movie.naver.com/movie/bi/mi/basic.naver?code=13558


In [7]:
# 날짜와 작성자 텍스트 추출 
a_list = soup.select('.author')
len(a_list)
a_list[0].text, a_list[0].next_sibling.next_sibling

('tlat****', '22.05.03')

In [8]:
# 모든 요소를 찾아서 2차원 리스트에 저장 
kid_list = []
base_url = 'https://movie.naver.com'

for i in range(len(a_list)):
    title = td_list[i].select_one('a').text
    score = td_list[i].select_one('em').text
    review = td_list[i].select_one('br').next_sibling.strip()
    link = base_url + td_list[i].select_one('a')['href']
    writer = a_list[i].text
    date = a_list[i].next_sibling.next_sibling
    
    # 1차원 리스트로 생성한 후 다시 리스트에 삽입      
    kid_list.append([title, score, review, link, writer, date])    

In [9]:
# 4) 2차원 리스트 => 데이타프레임 

df_kid = pd.DataFrame(kid_list, columns = ['title', 'score', 'review', 'link', 'writer', 'date'])
df_kid.sample(2)

,title,score,review,link,writer,date
7,키드,10,1921년도에 만들어졌다니 놀랍습니다!흑백영화는 처음보는데,https://movie.naver.com/movie/bi/mi/basic.nave...,yeyo****,21.03.01
9,키드,10,채플린만의 감동 연출법이 빛나는 영화. 진정한 휴머니스트. 희극의 천재.,https://movie.naver.com/movie/bi/mi/basic.nave...,got_****,21.02.23


# 범위를 지정해서 가져오기

In [10]:
# 각 페이지의 공통 주소 부분 
base_page = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
kid_list = []
base_url = 'https://movie.naver.com'

# 10페이지 저장 
for i in range(10):
    # 실제 페이지 주소 생성 
    url = base_page+str(i+1)
    res = requests.get(url)
    # 200 이면 성공 
    print(res, url) 
    # html 페이지를 텍스트로 저장 
    html_str = res.text
    
    # soup 객체화    
    soup = BeautifulSoup(html_str, 'html.parser')
    
    # 제목, 평점, 리뷰, 링크, 날짜와 작성자 텍스트 추출 
    td_list = soup.select('.title')
    a_list = soup.select('.author')

    # 모든 요소를 찾아서 2차원 리스트에 저장
    for j in range(len(a_list)):
        title = td_list[j].select_one('a').text
        score = td_list[j].select_one('em').text
        review = td_list[j].select_one('br').next_sibling.strip()
        link = base_url + td_list[j].select_one('a')['href']
        writer = a_list[j].text
        date = a_list[j].next_sibling.next_sibling

        # 1차원 리스트로 생성한 후 다시 리스트에 삽입      
        kid_list.append([title, score, review, link, writer, date])    
    

<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
<Response [200]> https://mov

In [12]:
# 2차원 리스트 => 데이타프레임 
df_kid2 = pd.DataFrame(kid_list, columns=['title', 'score', 'review', 'link', 'writer', 'date'])
df_kid2

,title,score,review,link,writer,date
0,키드,10,참으로 아름다운 영화다,https://movie.naver.com/movie/bi/mi/basic.nave...,tlat****,22.05.03
1,키드,10,50분 작품속에 인간과 사랑에 관한 모든 것이 담겨있다.,https://movie.naver.com/movie/bi/mi/basic.nave...,gkwl****,22.02.06
2,키드,10,짧고 굵게 행복하고 사랑스럽다,https://movie.naver.com/movie/bi/mi/basic.nave...,jsyb****,21.10.20
3,키드,8,(레미제라블의 장발장과 코제트간 관계가 다소간 롤리타적 색채를 띠는 걸 감안함)어쩌...,https://movie.naver.com/movie/bi/mi/basic.nave...,gold****,21.09.04
4,키드,10,과연 채플린의 영화이다! 이 참담한 포스트 코로나 시대에도 우리에게 웃음과 감동을 ...,https://movie.naver.com/movie/bi/mi/basic.nave...,rjkw****,21.03.23
...,...,...,...,...,...,...
95,키드,10,와 짱이다,https://movie.naver.com/movie/bi/mi/basic.nave...,jiwo****,10.08.20
96,키드,10,눈물겨운 어린 시절의 추억이 만들어낸 자화상적 이야기,https://movie.naver.com/movie/bi/mi/basic.nave...,hyuk****,10.08.11
97,키드,8,희극의제왕 찰리채플린!,https://movie.naver.com/movie/bi/mi/basic.nave...,powe****,10.06.01
98,키드,10,최고,https://movie.naver.com/movie/bi/mi/basic.nave...,wjdg****,10.05.03


In [13]:
# 마지막 페이지인지?  다음 버튼 소스 확인 

# 각 페이지의 공통 주소 부분 
base_page = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
kid_list = []
base_url = 'https://movie.naver.com'
page = 0
while(True):
    # 실제 페이지 주소 생성 
    url = base_page+str(page+1)
    res = requests.get(url)
    # 200 이면 성공 
    print(res, url) 
    # html 페이지를 텍스트로 저장 
    html_str = res.text
    
    # soup 객체화    
    soup = BeautifulSoup(html_str, 'html.parser')
    
    # 다음 버튼 찾기 
    next_btn = soup.find_all(class_='pg_next')
    # 다음버튼이 없다면 종료     
    if len(next_btn) == 0:
        break
    
    # 페이지 증감
    page += 1
    
print('테스트 종료')    

<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
<Response [200]> https://mov

In [14]:
# 각 페이지의 공통 주소 부분 
base_page = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
kid_list = []
base_url = 'https://movie.naver.com'
page = 0


while(True):
    # 실제 페이지 주소 생성 
    url = base_page+str(page+1)
    res = requests.get(url)
    # 200 이면 성공 
    print(res, url) 
    # html 페이지를 텍스트로 저장 
    html_str = res.text
    
    # soup 객체화    
    soup = BeautifulSoup(html_str, 'html.parser')
    
    # 제목, 평점, 리뷰, 링크, 날짜와 작성자 텍스트 추출 
    td_list = soup.select('.title')
    a_list = soup.select('.author')
    
    # 모든 요소를 찾아서 2차원 리스트에 저장
    for j in range(len(a_list)):
        title = td_list[j].select_one('a').text
        score = td_list[j].select_one('em').text
        review = td_list[j].select_one('br').next_sibling.strip()
        link = base_url + td_list[j].select_one('a')['href']
        writer = a_list[j].text
        date = a_list[j].next_sibling.next_sibling

        # 1차원 리스트로 생성한 후 다시 리스트에 삽입      
        kid_list.append([title, score, review, link, writer, date])

    
    # 다음 버튼 찾기 
    next_btn = soup.find_all(class_='pg_next')
    # 다음버튼이 없다면 종료     
    if len(next_btn) == 0:
        break
        
    # 페이지 증감
    page += 1
    
print('테스트 종료')        
    

<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
<Response [200]> https://mov

In [15]:
# 2차원 리스트 => 데이타프레임 
df_kid3 = pd.DataFrame(kid_list, columns=['title', 'score', 'review', 'link', 'writer', 'date'])
df_kid3.shape

(146, 6)

# 코드 개선 -> 함수로 

In [16]:
def save_df(base_page_url):
    # 각 페이지의 공통 주소 부분 
    base_page = base_page_url
    kid_list = []
    base_url = 'https://movie.naver.com'
    page = 0


    while(True):
        # 실제 페이지 주소 생성 
        url = base_page+str(page+1)
        res = requests.get(url)
        # 200 이면 성공 
        print(res, url) 
        # html 페이지를 텍스트로 저장 
        html_str = res.text

        # soup 객체화    
        soup = BeautifulSoup(html_str, 'html.parser')

        # 제목, 평점, 리뷰, 링크, 날짜와 작성자 텍스트 추출 
        td_list = soup.select('.title')
        a_list = soup.select('.author')

        # 모든 요소를 찾아서 2차원 리스트에 저장
        for j in range(len(a_list)):
            title = td_list[j].select_one('a').text
            score = td_list[j].select_one('em').text
            review = td_list[j].select_one('br').next_sibling.strip()
            link = base_url + td_list[j].select_one('a')['href']
            writer = a_list[j].text
            date = a_list[j].next_sibling.next_sibling

            # 1차원 리스트로 생성한 후 다시 리스트에 삽입      
            kid_list.append([title, score, review, link, writer, date])


        # 다음 버튼 찾기 
        next_btn = soup.find_all(class_='pg_next')
        # 다음버튼이 없다면 종료     
        if len(next_btn) == 0:
            break

        # 페이지 증감
        page += 1

    print('테스트 종료')   
    df = pd.DataFrame(kid_list, columns=['title', 'score', 'review', 'link', 'writer', 'date'])
    
    return df

In [17]:
# 고양이를 부탁해 
url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page='

In [18]:
# 고양이를 부탁해 영화 
base_page_url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page='
df_cat = save_df(base_page_url)

<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=1
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=2
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=3
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=4
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=5
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=6
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=7
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=8
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=9
<Response [200]> https://mov

# 네이버 책 검색 API

In [25]:
import os
import sys
import urllib.request
import json
import xmltodict
def book_api():
    # 신청한 아이디와 시크릿으로 변경 
    # client_id = "YOUR_CLIENT_ID"
    # client_secret = "YOUR_CLIENT_SECRET"
    client_id = "2bPOmVkKFf8eygLmGiMk"
    client_secret = "DllokD9yvw"

    # 검색할 단어를 입력하면 자동으로 주소 생성 
    book = input('검색어(책제목) =>')
    encText = urllib.parse.quote(book)
    url = "https://openapi.naver.com/v1/search/book?display=100&query=" + encText # json 결과
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    # 페이지가 정상적으로 접근되어진다면 저장 그렇지 않으면 에러메세지와 코드 출력 
    if(rescode==200):
        response_body = response.read()
    else:
        print("Error Code:" + rescode)
    
    book_list = json.loads(response_body)['items']
    
    # 데이타프레임으로 모두 저장
    
    df_book = pd.DataFrame(book_list)
    df_book['title'] = df_book['title'].apply(lambda x:x.replace('<b>','').replace('</b>',''))
    df_book = df_book[['title', 'author', 'price','discount', 'isbn']]
    return df_book


In [22]:
book_api()

,title,author,price,discount,isbn
0,어린 왕자 (Le Petit Prince),생텍쥐페리,9800,8820,8932917248 9788932917245
1,어린왕자 (초판본),생텍쥐페리,9800,8820,1159039690 9791159039690
2,어린왕자,생텍쥐페리|루이스 그리그,13000,11700,8959984396 9788959984398
3,어린 왕자,앙투안 드 생텍쥐페리,11000,9900,8964361946 9788964361948
4,2022년 어린왕자 마음의 눈으로 보이는 것들 탁상 달력,북엔 편집부,9800,4900,9529012071 8809529012073
...,...,...,...,...,...
95,어린 왕자 (중학교 영어로 다시 읽는 세계명작 07),생텍쥐페리,6500,5850,8957978062 9788957978061
96,어린 왕자,생텍쥐페리,7700,6930,8901215748 9788901215747
97,어린왕자 (밀레니엄북스 9),생텍쥐페리,12000,10800,8935910864 9788935910861
98,2021 유나 다이어리(어린왕자) (어린왕자),유나 편집부,15000,0,9479920204 8809479920206


In [26]:
def book_api_xml():
    # 신청한 아이디와 시크릿으로 변경 
    # client_id = "YOUR_CLIENT_ID"
    # client_secret = "YOUR_CLIENT_SECRET"
    client_id = "2bPOmVkKFf8eygLmGiMk"
    client_secret = "DllokD9yvw"

    # 검색할 단어를 입력하면 자동으로 주소 생성 
    book = input('검색어(책제목) =>')
    encText = urllib.parse.quote(book)
    # xml주소로 요청
    url = "https://openapi.naver.com/v1/search/book.xml?display=100&query=" + encText # json 결과
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    # 페이지가 정상적으로 접근되어진다면 저장 그렇지 않으면 에러메세지와 코드 출력 
    if(rescode==200):
        response_body = response.read()
#         print(response_body)
    else:
        print("Error Code:" + rescode)
    # 순서가 있는 
    # xml 텍스트 => orderedDict 
    orderedDict_data = xmltodict.parse(response_body)
#     print(orderedDict_data)
    dict_data = json.loads(json.dumps(orderedDict_data))
    book_list = dict_data['rss']['channel']['item']
#     print(book_list)
    df_book = pd.DataFrame(book_list)
    df_book['title'] = df_book['title'].apply(lambda x:x.replace('<b>','').replace('</b>',''))
    df_book = df_book[['title', 'author', 'price','discount', 'isbn']]
    return df_book

In [27]:
book_api_xml()

,title,author,price,discount,isbn
0,어린 왕자 (Le Petit Prince),생텍쥐페리,9800,8820,8932917248 9788932917245
1,어린왕자 (초판본),생텍쥐페리,9800,8820,1159039690 9791159039690
2,어린왕자,생텍쥐페리|루이스 그리그,13000,11700,8959984396 9788959984398
3,어린 왕자,앙투안 드 생텍쥐페리,11000,9900,8964361946 9788964361948
4,2022년 어린왕자 마음의 눈으로 보이는 것들 탁상 달력,북엔 편집부,9800,4900,9529012071 8809529012073
...,...,...,...,...,...
95,어린 왕자 (중학교 영어로 다시 읽는 세계명작 07),생텍쥐페리,6500,5850,8957978062 9788957978061
96,어린 왕자,생텍쥐페리,7700,6930,8901215748 9788901215747
97,어린왕자 (밀레니엄북스 9),생텍쥐페리,12000,10800,8935910864 9788935910861
98,2021 유나 다이어리(어린왕자) (어린왕자),유나 편집부,15000,0,9479920204 8809479920206
